In [15]:
import os
import json
import numpy as np

In [9]:
dataset_path = '../../'

meta_root_path = os.path.join(dataset_root, 'meta')
game_ids = next(os.walk(meta_root))[1]

for game_id in game_ids:
    meta_game_root_path = os.path.join(meta_root_path, game_id)
    game_samples = next(os.walk(meta_game_root_path))[2]

../../meta\beatsaber
['beatsaber_0.json', 'beatsaber_1.json', 'beatsaber_10.json', 'beatsaber_11.json', 'beatsaber_2.json', 'beatsaber_3.json', 'beatsaber_4.json', 'beatsaber_5.json', 'beatsaber_6.json', 'beatsaber_7.json', 'beatsaber_8.json', 'beatsaber_9.json']
../../meta\boxvr
['boxvr_0.json', 'boxvr_1.json', 'boxvr_2.json', 'boxvr_3.json', 'boxvr_4.json', 'boxvr_5.json']
../../meta\generic
['generic_0.json', 'generic_1.json', 'generic_2.json']
../../meta\jobsimulator
['jobsimulator_0.json', 'jobsimulator_1.json', 'jobsimulator_10.json', 'jobsimulator_11.json', 'jobsimulator_12.json', 'jobsimulator_13.json', 'jobsimulator_14.json', 'jobsimulator_2.json', 'jobsimulator_3.json', 'jobsimulator_4.json', 'jobsimulator_5.json', 'jobsimulator_6.json', 'jobsimulator_7.json', 'jobsimulator_8.json', 'jobsimulator_9.json']
../../meta\spacepiratetrainer
['spacepiratetrainer_0.json', 'spacepiratetrainer_1.json', 'spacepiratetrainer_2.json', 'spacepiratetrainer_3.json', 'spacepiratetrainer_4.json

In [101]:
import random

def chunks_per_ratio(num_frames, split, minC, maxC):
    f = int(num_frames * split)
    c = int(f / maxC)
    if f - (c * maxC) > minC:
        c += 1

    return f, c

def chunk_frames(num_frames, split_test = 0.15, split_validation = 0.20, min_chunk = 30, max_chunk = 300):
    num_chunks = int(num_frames/max_chunk)
    tot_frames = num_chunks * max_chunk
    if num_frames - tot_frames >= min_chunk:
        num_chunks += 1
        tot_frames = num_frames

    frames = [0,0,0]
    prefix = ['train_','val_', 'test_']
    frames[0], num_train_chunks = chunks_per_ratio(num_frames, 1.0 - (split_test + split_validation), min_chunk, max_chunk)
    frames[1], num_val_chunks = chunks_per_ratio(num_frames, split_validation, min_chunk, max_chunk)
    frames[2], num_test_chunks = chunks_per_ratio(num_frames, split_test, min_chunk, max_chunk)
    tot_frames = frames[0] + frames[1] + frames[2]

    rnd_distribution = [0] * num_train_chunks
    rnd_distribution.extend([1]* num_val_chunks)
    rnd_distribution.extend([2]* num_test_chunks)
    random.shuffle(rnd_distribution)

    chunk_out = list()
    for rnd_id in rnd_distribution:
        chunkSize = max_chunk
        if frames[rnd_id] < chunkSize:
            chunkSize = frames[rnd_id]
        frames[rnd_id] -= chunkSize
        
        chunk_out.append(f'{prefix[rnd_id]}{chunkSize}')

    return chunk_out

def split_source_file(bvh_source, split_key, out_path):
    

['train_300', 'train_300', 'train_300', 'train_300', 'train_300', 'train_300', 'train_300', 'test_300', 'test_300', 'train_300', 'val_300', 'test_300', 'val_300', 'train_300', 'val_300', 'train_300', 'val_300', 'train_300', 'train_300', 'val_300', 'train_300', 'test_300', 'train_300', 'train_300', 'train_300', 'train_300', 'train_300', 'val_186', 'train_81', 'test_64']
